In [7]:
import pandas as pd
import os

# ------------------------------
# 1. Configurações de pasta
# ------------------------------
data_folder = "../data"         # pasta onde estão os CSVs dos blocos
output_folder = "../outputs"    # pasta para salvar os pickles


# 2. Listar arquivos CSV
# ------------------------------
csv_files = [f for f in os.listdir(data_folder) if f.endswith(".csv")]

# ------------------------------
# 3. Loop para processar cada bloco
# ------------------------------
for csv_file in csv_files:
    bloco_nome = csv_file.replace(".csv", "")  # ex: SQN_102_B
    file_path = os.path.join(data_folder, csv_file)

    # ------------------ Ler CSV ------------------
    df = pd.read_csv(file_path, sep=";", header=None, low_memory=False)
    
    # ------------------ Tensões ------------------
    # linhas 33,34,36 -> índices 32,33,35; colunas D,E,F -> índices 3,4,5
    df_tensoes = df.loc[[32, 33, 35], [3, 4, 5]].copy()
    df_tensoes.columns = ["V1", "V2", "V3"]
    df_tensoes.index = ["Average value", "Max value", "Min value"]
    
    # Substituir vírgula por ponto e converter para float
    df_tensoes = df_tensoes.replace(",", ".", regex=True).astype(float)
    
    print("\n--- Tensões ---")
    display(df_tensoes)
    #print(df_tensoes.dtypes)
    
    # ------------------ Correntes ------------------
    # linhas 33..37 -> índices 32..36; colunas M,N,O -> índices 12,13,14
    df_correntes = df.loc[32:36, [12, 13, 14]].copy()
    df_correntes.columns = ["A1", "A2", "A3"]
    df_correntes.index = [
        "Average value",
        "Max value",
        "Max value recorded time",
        "Min value",
        "Min value recorded time",
    ]
    
    # separar linhas numéricas e de tempo
    numeric_rows = ["Average value", "Max value", "Min value"]
    time_rows = ["Max value recorded time", "Min value recorded time"]
    
    # converter valores numéricos
    df_values = df_correntes.loc[numeric_rows].replace(",", ".", regex=True).astype(float)
    
    # converter timestamps (formato ISO ou com underscore)
    df_times = df_correntes.loc[time_rows].apply(
        lambda col: pd.to_datetime(
            col.astype(str).str.replace("_", " ").str.strip("'\" "), 
            dayfirst=True,  # <-- aqui
            errors="coerce"
        )
    )
    
    # Formatar datas para DD/MM/YYYY HH:MM:SS
    df_times = df_times.apply(lambda col: col.dt.strftime("%d/%m/%Y %H:%M:%S"))
    
    #print("\n--- Correntes: valores numéricos ---")
    #display(df_values)
    #print(df_values.dtypes)
    
    #print("\n--- Correntes: timestamps ---")
    #display(df_times)
    #print(df_times.dtypes)
    
    # ------------------ Summary combinado (opcional) ------------------
    # convertendo para object para permitir tipos mistos
    summary = pd.concat([df_values.astype(object), df_times.astype(object)])
    summary = summary.loc[numeric_rows + time_rows]
    
    print("\n--- Summary combinado ---")
    display(summary)
    
    # ------------------ Potências ------------------
    # Ajuste os índices das colunas do CSV conforme necessário
    # Exemplo: P1,P2,P3,P total -> colunas 16,17,18,19
    df_pot = df.loc[[32, 33, 35], [22, 23, 24, 21]].copy()
    df_pot.columns = ["P1", "P2", "P3", "Ptot"]
    df_pot.index = ["Average value", "Max value", "Min value"]
    
    # Converter vírgula decimal para ponto e para float
    df_pot = df_pot.replace(",", ".", regex=True).astype(float)
    
    print("\n--- Potências (P1,P2,P3 e P total) ---")
    display(df_pot)
    #print(df_pot.dtypes)

    # ---------- Salvar em pickle ----------
    df_correntes.to_pickle(os.path.join(output_folder, f"{bloco_nome}_correntes.pkl"))
    df_tensoes.to_pickle(os.path.join(output_folder, f"{bloco_nome}_tensoes.pkl"))
    df_pot.to_pickle(os.path.join(output_folder, f"{bloco_nome}_potencias.pkl"))
    summary.to_pickle(os.path.join(output_folder, f"{bloco_nome}_datetime_correntes.pkl"))
    
  

    print(f"Arquivo salvo: {output_file}")


--- Tensões ---


,V1,V2,V3
Average value,222.0,221.0,223.0
Max value,235.0,235.0,235.0
Min value,213.0,211.0,213.0



--- Summary combinado ---


,A1,A2,A3
Average value,8.95,12.8,12.7
Max value,48.0,57.0,45.4
Min value,1.68,3.44,3.63
Max value recorded time,20/06/2023 23:28:00,28/06/2023 00:05:00,23/06/2023 02:13:00
Min value recorded time,25/06/2023 11:48:00,22/06/2023 13:22:00,25/06/2023 12:30:00



--- Potências (P1,P2,P3 e P total) ---


,P1,P2,P3,Ptot
Average value,1840.0,2700.0,2410.0,6950.0
Max value,10500.0,12100.0,9590.0,26000.0
Min value,-116.0,389.0,136.0,2920.0


Arquivo salvo: ../outputs\103_B_correntes.pkl
